In [1]:
# from agents.monte_carlo import MonteCarloAgent
from environment import TreasureCube

In [3]:
def test_cube(agent, max_episode=500, max_step=500):
    env = TreasureCube(max_step=max_step)

    for episode_num in range(0, max_episode):
        """Generate an episode using PI"""

        state = env.reset()
        terminate = False
        no_of_steps = 0
        episode_reward = 0

        # Start exploration
        while not terminate:
            action = agent.take_action(state)
            reward, terminate, next_state = env.step(action)
            episode_reward += reward
            # you can comment the following two lines, if the output is too much
            # env.render()  # comment
            # print(f'step: {t}, action: {action}, reward: {reward}')  # comment
            no_of_steps += 1
            agent.train(state, action, next_state, reward)
            state = next_state
        print(f'episode: {episode_num}, total_steps: {no_of_steps} episode reward: {episode_reward}')

In [2]:
import numpy as np
from collections import defaultdict

In [3]:
action_space = ['left', 'right', 'forward', 'backward', 'up', 'down'] # 0, 1, 2, 3, 4, 5
probabiity = 0.6
discount_factor = 0.99
SMALL_ENOUGH = 1e-3

In [4]:
def random_action(a, eps=0.1):
  # choose given a with probability 1 - eps + eps/4
  p = np.random.random()
  if p < (1 - eps):
    return a
  else:
    return np.random.choice(action_space)

In [5]:
def max_dict(d):
  # returns the argmax (key) and max (value) from a dictionary
  max_key = None
  max_val = float('-inf')
  for k, v in d.items():
    if v > max_val:
      max_val = v
      max_key = k
  return max_key, max_val

In [43]:
def play_game(grid, policy):
  # returns a list of states and corresponding returns
  # use an epsilon-soft policy
  s = tuple(grid.curr_pos)
  a = random_action(policy[s])

  # each triple is s(t), a(t), r(t)
  # but r(t) results from taking action a(t-1) from s(t-1) and landing in s(t)
  states_actions_rewards = [(s, a, 0)]
  while True:
    r, terminate, next_state = grid.step(a)
    s = tuple(grid.curr_pos)

    if terminate:
      states_actions_rewards.append((s, None, r))
      break
    else:
      a = random_action(policy[s]) # the next state is stochastic
      states_actions_rewards.append((s, a, r))

  # calculate the returns by working backwards from the terminal state
  G = 0
  states_actions_returns = []
  first = True
  for s, a, r in reversed(states_actions_rewards):
    # the value of the terminal state is 0 by definition
    # we should ignore the first state we encounter
    # and ignore the last G, which is meaningless since it doesn't correspond to any move
    if first:
      first = False
    else:
      states_actions_returns.append((s, a, G))
    G = r + discount_factor*G
  states_actions_returns.reverse() # we want it to be in order of state visited
  return states_actions_returns

In [7]:
def generate_random_policy(env):
    policy = {}

    for i in range(env.dim):
        for j in range(env.dim):
            for k in range(env.dim):
                if (i != 3 or j != 3 or k != 3):
                    policy[(i, j, k)] = np.random.choice(action_space)
    
    return policy

In [8]:
env = TreasureCube(max_step=500)

In [9]:
policy = generate_random_policy(env)

In [28]:
Q = {(i, j, k): actions_dict() for i in range(env.dim) for j in range(env.dim) for k in range(env.dim) if i != 3 or j != 3 or k != 3}
returns = defaultdict(list) # dictionary of state -> list of returns we've received

In [16]:
def actions_dict():
    return {key: 0 for key in action_space}

In [44]:
deltas = []

for t in range(5000):
    env.reset()
    # generate an episode using pi
    biggest_change = 0
    states_actions_returns = play_game(env, policy)

    # calculate Q(s,a)
    seen_state_action_pairs = set()
    for s, a, G in states_actions_returns:
        # check if we have already seen s
        # called "first-visit" MC policy evaluation
        sa = (s, a)
        if sa not in seen_state_action_pairs:
            old_q = Q[s][a]
            returns[sa].append(G)
            Q[s][a] = np.mean(returns[sa])
            biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
            seen_state_action_pairs.add(sa)
    deltas.append(biggest_change)

    # calculate new policy pi(s) = argmax[a]{ Q(s,a) }
    for s in policy.keys():
        a, _ = max_dict(Q[s])
        policy[s] = a

In [45]:
V = {}
for s in policy.keys():
  V[s] = max_dict(Q[s])[1]

In [46]:
V

{(0, 0, 0): -0.9245702659225568,
 (0, 0, 1): -0.9114403187841482,
 (0, 0, 2): -0.8383649859079964,
 (0, 0, 3): -0.5369781773659742,
 (0, 1, 0): -0.7536048975474179,
 (0, 1, 1): -0.5019514199973373,
 (0, 1, 2): -0.6802182223575067,
 (0, 1, 3): -0.1788943625840231,
 (0, 2, 0): -0.6028788975351574,
 (0, 2, 1): -0.28846700824753446,
 (0, 2, 2): -0.3288608856376815,
 (0, 2, 3): 0.05472291505032255,
 (0, 3, 0): -0.43183883658226113,
 (0, 3, 1): -0.1895803586464933,
 (0, 3, 2): -0.03685135614412332,
 (0, 3, 3): 0.21163668215329476,
 (1, 0, 0): -0.8768690753362349,
 (1, 0, 1): -0.9169315792352511,
 (1, 0, 2): -0.8400905047785941,
 (1, 0, 3): -0.5651838472369082,
 (1, 1, 0): -0.602952957283853,
 (1, 1, 1): -0.6076131449680926,
 (1, 1, 2): -0.8025662419552912,
 (1, 1, 3): -0.6924258424100578,
 (1, 2, 0): -0.3220223964538744,
 (1, 2, 1): -0.22114816659418385,
 (1, 2, 2): -0.18832144625493838,
 (1, 2, 3): 0.24662002761362384,
 (1, 3, 0): -0.26242612875935095,
 (1, 3, 1): 0.07122448839606955,
 (1, 

In [21]:
from pprint import pprint

In [48]:
pprint(Q)

{(0, 0, 0): {&#39;backward&#39;: -1.768804469437437,
             &#39;down&#39;: -1.2592844805141954,
             &#39;forward&#39;: -1.2648750928821215,
             &#39;left&#39;: -1.2971637320860223,
             &#39;right&#39;: -0.9245702659225568,
             &#39;up&#39;: -1.147477680947293},
 (0, 0, 1): {&#39;backward&#39;: -1.231379752646394,
             &#39;down&#39;: -1.7383941697877658,
             &#39;forward&#39;: -0.9114403187841482,
             &#39;left&#39;: -1.7701391878049482,
             &#39;right&#39;: -2.2177732034733815,
             &#39;up&#39;: -1.1176061196279212},
 (0, 0, 2): {&#39;backward&#39;: -1.380285147361645,
             &#39;down&#39;: -2.1294854352626693,
             &#39;forward&#39;: -1.965033600459244,
             &#39;left&#39;: -1.325217787873129,
             &#39;right&#39;: -1.3801728794747976,
             &#39;up&#39;: -0.8383649859079964},
 (0, 0, 3): {&#39;backward&#39;: -0.996950484157539,
             &#39;down&#39;: -1.

In [49]:
policy

{(0, 0, 0): &#39;right&#39;,
 (0, 0, 1): &#39;forward&#39;,
 (0, 0, 2): &#39;up&#39;,
 (0, 0, 3): &#39;right&#39;,
 (0, 1, 0): &#39;right&#39;,
 (0, 1, 1): &#39;right&#39;,
 (0, 1, 2): &#39;right&#39;,
 (0, 1, 3): &#39;right&#39;,
 (0, 2, 0): &#39;right&#39;,
 (0, 2, 1): &#39;right&#39;,
 (0, 2, 2): &#39;up&#39;,
 (0, 2, 3): &#39;right&#39;,
 (0, 3, 0): &#39;forward&#39;,
 (0, 3, 1): &#39;forward&#39;,
 (0, 3, 2): &#39;forward&#39;,
 (0, 3, 3): &#39;forward&#39;,
 (1, 0, 0): &#39;up&#39;,
 (1, 0, 1): &#39;up&#39;,
 (1, 0, 2): &#39;up&#39;,
 (1, 0, 3): &#39;backward&#39;,
 (1, 1, 0): &#39;up&#39;,
 (1, 1, 1): &#39;right&#39;,
 (1, 1, 2): &#39;forward&#39;,
 (1, 1, 3): &#39;backward&#39;,
 (1, 2, 0): &#39;right&#39;,
 (1, 2, 1): &#39;right&#39;,
 (1, 2, 2): &#39;up&#39;,
 (1, 2, 3): &#39;right&#39;,
 (1, 3, 0): &#39;up&#39;,
 (1, 3, 1): &#39;up&#39;,
 (1, 3, 2): &#39;forward&#39;,
 (1, 3, 3): &#39;forward&#39;,
 (2, 0, 0): &#39;backward&#39;,
 (2, 0, 1): &#39;up&#39;,
 (2, 0, 2): &#39;ba